In [2]:
import tqdm
import jieba
from tensorflow.keras.callbacks import TensorBoard
import kashgari
from kashgari.embeddings import BERTEmbedding         #Kashgari 内置了bert预训练语言模型处理模块
# 使用 embedding 初始化模型
from kashgari.tasks.classification import BiLSTM_Model
from tensorflow.python import keras
from kashgari.callbacks import EvalCallBack
import  tensorflow as tf
import os
def read_data_file(path):
    lines = open(path, 'r', encoding='utf-8').read().splitlines()
    x_list = []
    y_list = []
    for line in tqdm.tqdm(lines):
        rows = line.split('\t')
        if len(rows) >= 2:
            y_list.append(rows[0])
            x_list.append(list(jieba.cut('\t'.join(rows[1:]))))
        else:
            print(rows)
    return x_list, y_list

test_x, test_y = read_data_file('cnews/cnews.test.txt')
train_x, train_y = read_data_file('cnews/cnews.train.txt')
val_x, val_y = read_data_file('cnews/cnews.val.txt')
print(train_x[1])
print(train_y[1])

  0%|          | 0/10000 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\17801\AppData\Local\Temp\jieba.cache
DEBUG:jieba:Loading model from cache C:\Users\17801\AppData\Local\Temp\jieba.cache
Loading model cost 0.575 seconds.
DEBUG:jieba:Loading model cost 0.575 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.
100%|██████████| 5000/5000 [00:12<00:00, 385.26it/s]


['蚂蚁的连击，这个么，单纯懒得去验证，而且我感觉近战用这个技能才更需要测试。圣堂的贯穿式攻击、tb龙骑变身后的远程溅射，这个，理论上应该是不算在以上四个等级的攻击特效里面的，所以属于完全不受影响的技能。依然那句话，ak不到无法测试……攻击特效之间的相互影响（未测试，仅提出想法）： 不知道有多少人看过以前那个法球效果表，dota正常版里法球绝大部分是无法叠加的，优先级一般是自动施放法球>吸血面具>其他法球，其他法球中的优先级顺序为物品栏中靠前的法球优先，最后获得的法球优先。唯一一个例外是远程英雄的冰眼，以及龙骑三段变身的霜冻攻击，这两个是可以和吸血面具叠加的。除此之外，冰眼还有一个十分强大的功能，就是强化法球。具体内容记不太清楚了，只晓得冰眼和散夜或者雷锤同时带身上，可以提高触发概率。同时冰眼的效果会受到英雄本身近战/远程状态的影响，tb巨魔龙骑这种会变来变去的英雄在某个状态捡起冰眼之后变成另一个状态，冰眼的状态会保持原本的近战或者远程状态。imba版本里虽然法球之间的冲突已经基本全部消失了，可是对于冰眼这么奇妙欢乐的玩意儿，说不准还真的挺有内容的。更多衍生的想法：imba里面的攻击特效已经非常多了，我发现米米亚也开始逐步增加施法特效了。先是火女宙斯，然后卡拉波和60智力法杖，现在蓝胖被解禁，冰甲加入了新的im特效。我建议可以加入更多施法特效的物品，目前只有冰甲和卡拉波，而且效果也不怎么看得出来，永恒长夜法杖又是个基本没人出的货。法系装备略贵，也略少。另一方面，我个人玩imba纯粹为了娱乐，所以也不在乎平衡不平衡的问题，一切归结于脸。所以我觉得imba如果依然是想要往娱乐的方向发展的话，可以将随机性的成分继续放大。幽鬼折射就是个很好的例子，从百分比的硬免伤，变成了几率性的全面伤，人马一个双刃剑弹死自己的现象屡见不鲜。所以人马刚毅不屈和死灵龙守墓人斗篷之类的技能，我觉得也可以往几率性全免伤上面靠。而同时，很多ak技能废物自己就基本成渣的英雄，可以考虑加强一下。比如谜团之流。而那些大招十分强力但是其他技能就完全不够看的英雄也可以考虑改改，典型的例子就是先知。另外召唤师的解禁让我感觉到操作流英雄的归来，虽然我自己不会用，但是我希望召唤师可以是一个高手拯救世界、新手平淡无奇的英雄，并且期待地卜师强势回归。']
['商瑞华', '首战', '复仇', '心切', ' ', '

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1491)]            0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 1491, 100)         16815700  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1487, 128)         64128     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 16,888,734
Trainable params: 16,888,734
Non-trainable params: 0
_______________________________________________

InvalidArgumentError: You must feed a value for placeholder tensor 'input' with dtype float and shape [?,1491]
	 [[{{node input}}]]

In [2]:
# -*- coding: utf-8 -*-
import jieba
import jieba.posseg as pseg
import datetime
import sys


#词性标注，nr为人名
def getFirstName(messageContent):
    words = pseg.cut(messageContent)
    for word, flag in words:
        if flag == 'nr'and len(word)>1:#单字姓名去掉
            return word

    return False

def getAllName(messageContent):
    words = pseg.cut(messageContent)
    names = []
    for word, flag in words:
        print('%s,%s' %(word,flag))
        if flag == 'nr':#人名词性为nr
            names.append(word)
    return names

#修改停用词集合中所有词性为名词，大部分为名词
def alterWordTagToX(list):
    for x in list:
        jieba.add_word(x, tag='n')

def LoadStopWord(StopWordFileName):
    StopWord_file = open(StopWordFileName, 'r', encoding='utf-8')
    StopWordList = []

    for line in StopWord_file.readlines():
        StopWordList.append(line.strip('\n'))

    set(StopWordList)
    StopWord_file.close()
    alterWordTagToX(StopWordList)

def main():
    #加载停用词词典文件
    LoadStopWord('stop_words/stop_words.txt')
    #
    # input_file_name = sys.argv[1]
    # output_file_name = 'name_'+ input_file_name
    # print(input_file_name)
    # print(output_file_name)
    # begin = datetime.datetime.now()
    # #单机并行分词
    # jieba.enable_parallel(8)
    # input_file = open(input_file_name, 'r', encoding='utf-8')
    # output_file = open(output_file_name, 'w')
    #
    # for line in input_file:
    #     temp = line.split('\t')
    #     if len(temp)!=4:
    #         continue
    #     name = getFirstName(temp[1])
    #
    #     if name != False:
    #         #print(name)姓名作为一行中的一个字段，其他为你需要的字段
    #         time = str(temp[3]).strip('\n')
    #         output_file.write(temp[0] + ','+ name + ','+ '\n')
    #     else:
    #         continue
    #
    # end = datetime.datetime.now()
    # print((end - begin).seconds)

#单元测试代码
    names = getAllName('我老公宝贝叫王宁,尊敬的王大力,CCHHKK旗舰店,尊敬的铁路客服人员李天，冯达辉')
    print(names)
    print(getFirstName('尊敬的铁路客服人员李天'))
    # output_file.close()
    # input_file.close()


if __name__ =='__main__':
    main()

我,n
老公,nr
宝贝,nr
叫,v
王宁,nr
,,x
尊敬,v
的,uj
王,n
大力,n
,,x
CCHHKK,eng
旗舰店,v
,,x
尊敬,v
的,n
铁路,n
客服,n
人员,n
李天,nr
，,x
冯达辉,nr
['老公', '宝贝', '王宁', '李天', '冯达辉']
李天
